### Install requires packages

In [1]:
!pip install langchain pypdf pypdf2 qdrant-client sentence_transformers transformers tiktoken python-dotenv qdrant_client --quiet
!pip install -U langchain-community --quiet
!pip install qdrant_client
!pip install ctransformers
!pip install -U langchain-qdrant

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13

### Import the required packages

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Qdrant
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import TokenTextSplitter
from qdrant_client import models, QdrantClient
import os

In [ ]:
import warnings
# Ignore all warnings
warnings.filterwarnings('ignore')

### Data loading

In [ ]:
# Load Kenya National Fomularly Data
presc_data = "/content/drive/MyDrive/ML projects/pharmChat/Data"

loader = DirectoryLoader(presc_data, glob="Kenya_National_Medicines_Formulary_2023_1st_Edition.pdf", show_progress=True, loader_cls=PyPDFLoader)

documents = loader.load()

100%|██████████| 1/1 [00:43<00:00, 43.17s/it]


### Data Preprocessing

In [ ]:
# Load token text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)

#tokenize the data
tokenized_docs = text_splitter.split_documents(documents)

### Qdrant Vector Database Confguration

In [ ]:
# Qdrant cloud url
qdrant_cloud = 'https://56869e5c-b05f-40a7-940e-101b0310f818.us-east4-0.gcp.cloud.qdrant.io:6333'
qdrant_api_key = os.getenv('QDRANT_API_KEY')

In [ ]:
# Create an in-memory Qdrant instance
client = QdrantClient(":memory:")

In [ ]:
# Load pubmedbert embeddings
embeddings = SentenceTransformerEmbeddings(model_name = "NeuML/pubmedbert-base-embeddings")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_downlo

In [ ]:
# # # Embed tokens and store them in the vector db
# qdrant = Qdrant.from_documents(
#     tokenized_docs,
#     embeddings,
#     url=qdrant_cloud,
#     api_key=qdrant_api_key,
#     prefer_grpc = False,
#     collection_name = "medical_fomularly"
# )

In [ ]:
# # # Embed tokens and store them in the vector db
qdrant = Qdrant.from_documents(
    tokenized_docs,
    embeddings,
    url=qdrant_cloud,
    api_key="input_key",
    prefer_grpc = False,
    collection_name = "medical_fomularly"
)

In [ ]:
from langchain_qdrant import Qdrant

In [ ]:
client = QdrantClient(
    url=qdrant_cloud,
    api_key="input_key",
    prefer_grpc=False
)

db = Qdrant(client=client, embeddings=embeddings, collection_name="medical_fomularly")
# db = QdrantClient(api_key=qdrant_api_key)

In [ ]:
# Test query
query = "what is Halothane?"

# Seerch in info in the database vectors
docs = db.similarity_search_with_score(query=query, k=3)

# show retrieved tokens
for i in docs:
    doc, score = i
    print({"score": score, "Content":doc.page_content, "metadata":doc.metadata})

## Retrieval Augmented Generation

In [ ]:
# Load required Prerequisites
from langchain import PromptTemplate
from dotenv import load_dotenv
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA

In [ ]:
# Load Pretrained Local Model
local_llm = "/content/drive/MyDrive/ML projects/pharmChat/Model/meditron-7b.Q5_K_M.gguf"

In [ ]:
# Configure CUDA
if torch.cuda.is_available():
    threads = int(os.cpu_count() / 4)
else:
    threads = int(os.cpu_count() / 2)

# Configure Output tokens
config = {
    'max_new_tokens': 1024,
    'context_length': 2048,
    'repetition_penalty': 1.1,
    'temperature': 0.1,
    'top_k': 50,
    'top_p': 0.9,
    'stream': True,
    'threads': threads
}

print(config)


{'max_new_tokens': 1024, 'context_length': 2048, 'repetition_penalty': 1.1, 'temperature': 0.1, 'top_k': 50, 'top_p': 0.9, 'stream': True, 'threads': 2}


In [ ]:
# Load HuggingFace API
token = os.getenv("HUGGINGFACE_API_TOKEN")

In [ ]:
# LLM Transformer Configuration
llm = CTransformers(
    model=local_llm,
    model_type="llama",
    lib="avx2",
    api_key=token,
    **config
)

In [ ]:
# Print GPU threads
# Using 2 to reduce Memory overload
print(f"Using {config['threads']} threads for processing.")

Using 2 threads for processing.


In [ ]:
# Configure Response Output Template

prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
# Initialize PromptTemplate and pass the template
prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])

In [ ]:
# Configure RAG
retriever = db.as_retriever(search_kwargs={"k":1})
chain_type_kwargs = {"prompt": prompt}
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True, chain_type_kwargs=chain_type_kwargs, verbose=True)


In [ ]:
# Sample question
query = "What is Halothane"

In [ ]:
# Pass the sample question to the the RAG
response = qa(query)

# Print response
print(response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...



> Finished chain.
{'query': 'What is Halothane', 'result': 'Halothane (Fluothane) is a  volatile\nanaesthetic agent that has been used since 1956, \nand is still in clinical use as of today. Halothane \nis an alkyl halide with the formula C2F4BrCH3. It \nis a colourlessensuresulifies from mimparts with the only requires toxic and other volat room temperature (halothers as a colourless frequently referred to produce the simplest molecular volume is structuronset the  has a molecannotates from bubiquenforces, like En route into air) but it is used inhappears its chemical structure consists of halogeniclends with other halogenousingests with one carbonate.\nis colourless potenters as ana colourless frequently induces a liquid form a powerful solubiquidupharmescapitulizes like  belongs to mimproved and dipsychlooses fluorallylays to the molecularlysometimes is structuranestabranges its formula is a volatoms inhibitself produces a very easily evaporates into an agent with ', 'source_docume

In [ ]:
answer = response['result']
source_document = response['source_documents'][0].page_content
doc = response['source_documents'][0].metadata['source']
doc_page = response['source_documents'][0].metadata['page']

In [ ]:
# Configure pretty print
import pprint
pp = pprint.PrettyPrinter(indent=4, width=80, depth=None, compact=False)


In [ ]:
# Print answer
pp.pprint(answer)

('Halothane (Fluothane) is a  volatile\n'
 'anaesthetic agent that has been used since 1956, \n'
 'and is still in clinical use as of today. Halothane \n'
 'is an alkyl halide with the formula C2F4BrCH3. It \n'
 'is a colourlessensuresulifies from mimparts with the only requires toxic and '
 'other volat room temperature (halothers as a colourless frequently referred '
 'to produce the simplest molecular volume is structuronset the  has a '
 'molecannotates from bubiquenforces, like En route into air) but it is used '
 'inhappears its chemical structure consists of halogeniclends with other '
 'halogenousingests with one carbonate.\n'
 'is colourless potenters as ana colourless frequently induces a liquid form a '
 'powerful solubiquidupharmescapitulizes like  belongs to mimproved and '
 'dipsychlooses fluorallylays to the molecularlysometimes is '
 'structuranestabranges its formula is a volatoms inhibitself produces a very '
 'easily evaporates into an agent with ')


In [ ]:
# Print source document
print(source_document)

unexplained pyrexia or jaundice following previous 
exposure to halothane.
Adverse	effects:	Bradycardia, respiratory depression, 
arrhythmias, hepatitis (may be fatal)
Interactions with other medicines (*indicates serious): 
Amitriptyline, *chlorpromazine, diazepam, enalapril, 
epinephrine (adrenaline), *fluphenazine, furosemide, 
*haloperidol, isoniazid, *levodopa, spironolactone, 
suxamethonium, vancomycin, vecuronium, *verapamilNotes
 » Does not augment salivary or bronchial 
secretions
 » Cardiac sensitization to the effects of 
catecholamines
Isoflurane	
ATC code:  N01AB06
Inhalation, 100% inhalation vapour liquid, 250 mL, LOU 4
Indications and dose
Induction of anaesthesia (in oxygen or nitrous oxide–
oxygen) (but indication not recommended in infants and 
children of all ages):  Agent is pungent, hence risk of 
cough and bronchospasms.
Adult
By inhalation: Initially 0.5%, increased to 3%, adjusted 
according to response, administered using specifically 
calibrated vaporizer


In [ ]:
# Print sourcr page number
print(doc_page)

19
